# num_workers

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

from torch.utils.tensorboard import SummaryWriter   # 用来将网络数据发送到tensorboard中

import time
import pandas as pd
import json
from IPython.display import clear_output

from collections import OrderedDict
from collections import namedtuple
from itertools import product

In [2]:
class RunBuilder():
    @staticmethod          # 静态函数可以直接用类名调用，不需要创建类的实例
    def get_runs(params):
        Run = namedtuple("Run", params.keys())   # 用于构建带名字的元组，名字为"Run"
        
        runs = []
        for v in product(*params.values()):      # 对于所有超参数的组合
            runs.append(Run(*v))                 # 构建一个带名元组
        
        return runs

In [3]:
class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_strat_time = None
        
        self.network = None
        self.loader = None
        self.tb = None       # summarywriter
        
    def begin_run(self, run, network, loader):        # 每次运行开始调用
        self.run_start_time = time.time()
        
        self.run_params = run      # run代表所有超参数的元组
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')    # 为本次运行起名
        
        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)
        
        self.tb.add_image('image', grid)
        # self.tb.add_graph(self.network, images)
        self.tb.add_graph(
            self.network,
            images.to(getattr(run, 'device', 'cpu'))
        )
        
    def end_run(self):           # 每次运行结束调用
        self.tb.close()
        self.epoch_count = 0     # 初始化epoch计数
        
    def begin_epoch(self):         # 每个epoch开始时
        self.epoch_start_time = time.time()  # 设置开始时间
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        
    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time
        
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)
        
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
            
        results = OrderedDict()            # 有序字典存放运行结果
        # 把需要的数据加入字典
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        for k, v in self.run_params._asdict().items(): # 对于本次运行的所有超参数，转换成字典，并提取键值
            results[k] = v         # 加入results
            
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')  # 利用pandas将结果格式化输出
        
        clear_output(wait=True)    # 清空Jupyter notebook当前输出
        display(df)                # 在Jupyter notebook中输出结果

    def track_loss(self, loss):
        self.epoch_loss += loss.item() * self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)
        
    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self, fileName):

        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns'
        ).to_csv(f'{fileName}.csv')

        with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [4]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)     # Linear = fully connected(fc) = dense
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # (1) input layer
        t = t
        
        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)                   # 卷积层中不包含激活函数，需要手动添加
        t = F.max_pool2d(t, kernel_size=2, stride=2)     # 池化操作（没有权重的函数不能称为层）
        
        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden linear layer
        t = t.reshape(-1, 12*4*4)       # 必须手动flatten
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer
        t = self.out(t)
        # t = F.softmax(t, dim=1)           # 不直接在forward中用softmax，而是在训练过程中用cross-entropy损失函数计算loss，其中自带softmax
        
        return t
    
    
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)


In [5]:
params = OrderedDict(
    lr = [0.01],
    batch_size = [100, 1000, 10000],
    num_workers = [0, 1, 2, 4, 8, 16]                    
)

m = RunManager()
for run in RunBuilder.get_runs(params):

    network = Network()
    loader = DataLoader(train_set, batch_size=run.batch_size, num_workers=run.num_workers)
    optimizer = optim.Adam(network.parameters(), lr = run.lr)

    m.begin_run(run, network, loader)
    for epoch in range(1):
        m.begin_epoch()
        for batch in loader:
            images, labels = batch      # get batch
            preds = network(images)     # pass batch
            loss = F.cross_entropy(preds, labels)   # caculate loss
            optimizer.zero_grad()       # zero gradients
            loss.backward()             # calculate gradients
            optimizer.step()            # update weights

            m.track_loss(loss)
            m.track_num_correct(preds, labels)
        m.end_epoch()
    m.end_run()
m.save("results")

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers
0,1,1,0.556357,0.787617,10.422159,10.680790,0.01,100,0
1,2,1,0.610789,0.766933,9.547362,12.108568,0.01,100,1
2,3,1,0.554689,0.793750,11.989568,15.615613,0.01,100,2
3,4,1,0.577062,0.784233,13.182068,18.154399,0.01,100,4
4,5,1,0.534990,0.798900,17.487153,25.900861,0.01,100,8
5,6,1,0.560213,0.788883,26.062017,42.994734,0.01,100,16
6,7,1,0.912553,0.651350,14.614140,15.822467,0.01,1000,0
7,8,1,1.038551,0.594583,13.564002,17.623340,0.01,1000,1
8,9,1,1.002503,0.610733,11.263471,16.969113,0.01,1000,2
9,10,1,0.951017,0.635950,13.607588,21.004122,0.01,1000,4


# Moving to GPU

In [6]:
for name, param in network.named_parameters():
    print(name, "\t\t\t", param.shape)

conv1.weight 			 torch.Size([6, 1, 5, 5])
conv1.bias 			 torch.Size([6])
conv2.weight 			 torch.Size([12, 6, 5, 5])
conv2.bias 			 torch.Size([12])
fc1.weight 			 torch.Size([120, 192])
fc1.bias 			 torch.Size([120])
fc2.weight 			 torch.Size([60, 120])
fc2.bias 			 torch.Size([60])
out.weight 			 torch.Size([10, 60])
out.bias 			 torch.Size([10])


In [7]:
for name, param in network.named_parameters():
    print(param.device, name)

cpu conv1.weight
cpu conv1.bias
cpu conv2.weight
cpu conv2.bias
cpu fc1.weight
cpu fc1.bias
cpu fc2.weight
cpu fc2.bias
cpu out.weight
cpu out.bias


In [9]:
torch.cuda.is_available()

False

# Using the GPU: Test

In [3]:
params = OrderedDict(
    lr = [0.01],
    batch_size = [100, 1000, 10000],
    num_workers = [0, 1, 2, 4, 8, 16]                    
)

m = RunManager()
for run in RunBuilder.get_runs(params):


    device = torch.device('cuda')
    network = Network().to(device)
#     network = Network()
    loader = DataLoader(train_set, batch_size=run.batch_size, num_workers=run.num_workers)
    optimizer = optim.Adam(network.parameters(), lr = run.lr)
    
    

    m.begin_run(run, network, loader)
    for epoch in range(1):
        m.begin_epoch()
        for batch in loader:
            images, labels = batch      # get batch
            images.to(device)
            labels.to(device)
            preds = network(images)     # pass batch
            loss = F.cross_entropy(preds, labels)   # caculate loss
            optimizer.zero_grad()       # zero gradients
            loss.backward()             # calculate gradients
            optimizer.step()            # update weights

            m.track_loss(loss)
            m.track_num_correct(preds, labels)
        m.end_epoch()
    m.end_run()
m.save("results")

NameError: name 'RunManager' is not defined